In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from functools import reduce

import sys
import itertools
# Add the project root directory to the system path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from trading_system_py.retriever.import_data_from_object import ImportDataFromObject
from trading_system_py.analysis.portfolio.portfolio import Portfolio
from trading_system_py.analysis.strategy.strategy import Strategy

data_path = os.path.abspath(os.path.join(os.getcwd(), '../data'))


## TRADING SYSTEM

### Cos’è un trading system?
- Codifica di una strategia di trading in un linguaggio di programmazione.  
- Genera automaticamente segnali di acquisto e vendita.

### Vantaggi
- **Verificabilità**: Test delle prestazioni passate.  
- **Quantificazione**: Profilo realistico di rischio e rendimento.  
- **Consistenza**: Regole uniformi per ogni operazione.  
- **Estensibilità**: Applicabile su più mercati.  
- **Capitalizzazione corretta**: Uso ottimale del capitale.

### Sviluppo e valutazione
- **Ottimizzazione in-sample**: Test sui dati storici.  
- **Simulazione out-of-sample**: Verifica su dati non inclusi nel training.  
- **Metriche di performance**:  
  - Net Profit, Drawdown, Average Trade, Percent Profitable.  

---

## STRATEGIE DI TRADING

### Trend follower
- Strategie che seguono la direzione dominante del mercato.  
- **Strumenti principali**: Medie mobili, ROC, MACD.  
- **Esempio**: Incrocio tra medie mobili lente e veloci.

### Contrarian
- Strategie che vanno contro il trend dominante.  
- **Strumenti principali**: RSI, Bande di Bollinger.  
- **Esempio**:  
  - **Put/Call Ratio**: Rapporto tra opzioni put e call, indicatore del sentiment del mercato.  

In [ ]:
# Generazione di dati fittizi per due asset
def generate_dummy_data(start_date, end_date, isin):
    """
    Genera dati storici fittizi per un asset.

    Parametri:
    - start_date (str): Data di inizio nel formato 'YYYY-MM-DD'.
    - end_date (str): Data di fine nel formato 'YYYY-MM-DD'.
    - isin (str): Codice ISIN dell'asset (utile per identificare i dati).

    Ritorna:
    - data (pd.DataFrame): DataFrame con dati storici fittizi.
    """
    # Generazione delle date
    dates = pd.date_range(start=start_date, end=end_date, freq='B')  # Giorni lavorativi

    # Generazione random walk per i prezzi di chiusura
    # np.random.seed(42)  # Per risultati riproducibili
    price = 100 + np.cumsum(np.random.randn(len(dates)))  # Random walk

    # Creazione DataFrame
    data = pd.DataFrame(index=dates)
    data['Close'] = price
    data['Open'] = data['Close'] + np.random.randn(len(dates))
    data['High'] = data[['Open', 'Close']].max(axis=1) + np.abs(np.random.randn(len(dates)))
    data['Low'] = data[['Open', 'Close']].min(axis=1) - np.abs(np.random.randn(len(dates)))
    data['Volume'] = np.random.randint(1000, 10000, size=len(dates))

    # Aggiunta del codice ISIN (opzionale)
    data['ISIN'] = isin

    return data

# Genera dati fittizi per 'ISIN_1' e 'ISIN_2'
start_date = '2020-01-01'
end_date = '2024-10-30'

# Crea il dizionario 'data' come specificato
isin_list = ['isin_1', 'isin_2', 'isin_3']
data_dict = {k: {'History': generate_dummy_data(start_date, end_date, k)} for k in isin_list}

In [ ]:
data_object = ImportDataFromObject(path=data_path, filename='sample_30_min.pickle')
data_dict= data_object.data

In [ ]:
max_sharpe_weigths = {'AAPL': 0.15,
  'MSFT': 0.19,
  'AMZN': 0.01,
  'GOOGL': 0.0,
  'TSLA': 0.08,
  'JNJ': 0.0,
  'PG': 0.0,
  'KO': 0.0,
  'XOM': 0.0,
  'JPM': 0.08,
  'WMT': 0.15,
  'V': 0.0,
  'MCD': 0.08,
  'INTC': 0.0,
  'LQD': 0.0,
  'TLT': 0.25,
  'BND': 0.0,
  'EMB': 0.0,
  'HYG': 0.0}

portfolio = Portfolio(data=data_dict, init_cash=100000, fee_plus=0.26, broker_fee=0.0, weights=max_sharpe_weigths)

In [ ]:
portfolio.cash_allocation

In [ ]:
# portfolio.get_price(isin='AAPL', date='2023-01-02')

In [ ]:
from trading_system_py.analysis.strategy.buy_and_hold import BuyAndHold
from trading_system_py.analysis.strategy.rsi_over_bought_sold_strategy import RSIOverboughtOversoldStrategy 
from trading_system_py.analysis.strategy.bollinger_bands_strategy import BollingerBandsStrategy 
from trading_system_py.analysis.strategy.lstm_strategy import LSTMNeuralNetworkStrategy
from trading_system_py.analysis.strategy.moving_average_cross_strategy import MovingAverageCrossStrategy
from trading_system_py.utils.technical_indicators import TechnicalIndicators
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Flatten

# Strategie senza docstring

# Strategy RSI settimanale
class WeeklyRSIStrategy(Strategy):
    def __init__(self, rsi_period=2, overbought=70, oversold=30):
        self.rsi_period = rsi_period
        self.overbought = overbought
        self.oversold = oversold

    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0
        data['weekly_RSI'] = TechnicalIndicators.calculate_indicator('RSI', data['Close'], timeperiod=self.rsi_period)
        
        signals['signal'] = np.where(data['weekly_RSI'] < self.oversold, 1.0, 
                                     np.where(data['weekly_RSI'] > self.overbought, -1.0, 0.0))
        
        # Manteniamo la posizione fino al prossimo segnale di RSI
        signals['positions'] = signals['positions'].ffill().fillna(0)
        
        return signals
    

# Strategy "Turn of the Month"
class TurnOfTheMonthStrategy(Strategy):
    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        signals['signal'] = np.where((data.index.day <= 3), 1.0, 0.0)
        signals['signal'] = np.where((data.index.day >= 27), -1.0, signals['signal'])
        
        # Manteniamo la posizione fino al successivo segnale di fine mese
        signals['positions'] = signals['positions'].ffill().fillna(0)
        
        return signals

# Strategy basata sulla volatilità
class VolatilityBasedStrategy(Strategy):
    def __init__(self, lookback_period=20, volatility_threshold=0.02):
        self.lookback_period = lookback_period
        self.volatility_threshold = volatility_threshold

    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0
        data['volatility'] = data['Close'].rolling(window=self.lookback_period).std()
        
        signals['signal'] = np.where(data['volatility'] > self.volatility_threshold, 1.0, -1.0)
        
        # La posizione segue il segnale sulla volatilità
        signals['positions'] = signals['signal'].diff().fillna(0)
        
        return signals
    
# Russell Rebalancing Strategy
class RussellRebalancingStrategy(Strategy):
    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        # Segnale di acquisto il 23 giugno e di vendita il primo giorno di luglio
        rebalance_day = (data.index.month == 6) & (data.index.day >= 23)
        rebalance_dates = data.index[rebalance_day]

        if not rebalance_dates.empty:
            first_rebalance_day = rebalance_dates[0]
            signals.loc[first_rebalance_day, 'signal'] = 1.0
            first_july_day = data[(data.index.month == 7)].index[0]
            signals.loc[first_july_day, 'signal'] = -1.0

        # Manteniamo la posizione fino al cambio del segnale
        signals['positions'] = signals['signal']
        return signals


# Rubber Band Strategy
class RubberBandStrategy(Strategy):
    def __init__(self, lookback_period=5, atr_multiplier=2.5):
        self.lookback_period = lookback_period
        self.atr_multiplier = atr_multiplier

    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        data['ATR'] = TechnicalIndicators.calculate_indicator('ATR', data['High'], data['Low'], data['Close'], timeperiod=self.lookback_period)
        data['5d_high'] = data['High'].rolling(window=self.lookback_period).max()
        data['lower_band'] = data['5d_high'] - (self.atr_multiplier * data['ATR'])
        
        signals['signal'] = np.where(data['Close'] < data['lower_band'], 1.0, 0.0)
        
        # Manteniamo la posizione fino a che il prezzo non supera la banda inferiore
        signals['positions'] = signals['signal']
        return signals


# MFI Indicator Strategy
class MFIIndicatorStrategy(Strategy):
    def __init__(self, mfi_period=2, mfi_threshold=10):
        self.mfi_period = mfi_period
        self.mfi_threshold = mfi_threshold

    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        data['MFI'] = TechnicalIndicators.calculate_indicator('MFI', data['High'], data['Low'], data['Close'], data['Volume'], timeperiod=self.mfi_period)
        
        signals['signal'] = np.where(data['MFI'] < self.mfi_threshold, 1.0, 0.0)
        signals['exit'] = np.where(data['Close'] > data['High'].shift(1), -1.0, 0.0)
        signals['signal'] = signals['signal'] + signals['exit']
        
        # Manteniamo la posizione fino al segnale di uscita
        signals['positions'] = signals['signal']
        return signals


# RSI and Bollinger Bands Strategy
class RSIandBollingerBandsStrategy(Strategy):
    def __init__(self, rsi_period=14, rsi_overbought=70, rsi_oversold=30, bb_period=20, bb_dev=2):
        self.rsi_period = rsi_period
        self.rsi_overbought = rsi_overbought
        self.rsi_oversold = rsi_oversold
        self.bb_period = bb_period
        self.bb_dev = bb_dev

    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        data['RSI'] = TechnicalIndicators.calculate_indicator('RSI', data['Close'], timeperiod=self.rsi_period)
        upper_band, middle_band, lower_band = TechnicalIndicators.calculate_indicator(
            'BBANDS', data['Close'], timeperiod=self.bb_period, nbdevup=self.bb_dev, nbdevdn=self.bb_dev)
        
        signals['signal'] = np.where((data['Close'] < lower_band) & (data['RSI'] < self.rsi_oversold), 1.0, 
                                     np.where((data['Close'] > upper_band) & (data['RSI'] > self.rsi_overbought), -1.0, 0.0))
        
        # Manteniamo la posizione fino al segnale opposto
        signals['positions'] = signals['signal']
        return signals
    

# Multi-Timeframe RSI Strategy
class MultiTimeframeRSIStrategy(Strategy):
    def __init__(self, short_rsi=7, medium_rsi=14, long_rsi=28):
        self.short_rsi = short_rsi
        self.medium_rsi = medium_rsi
        self.long_rsi = long_rsi

    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        data['short_rsi'] = TechnicalIndicators.calculate_indicator('RSI', data['Close'], timeperiod=self.short_rsi)
        data['medium_rsi'] = TechnicalIndicators.calculate_indicator('RSI', data['Close'], timeperiod=self.medium_rsi)
        data['long_rsi'] = TechnicalIndicators.calculate_indicator('RSI', data['Close'], timeperiod=self.long_rsi)

        signals['signal'] = np.where((data['short_rsi'] < 30) & (data['medium_rsi'] < 30) & (data['long_rsi'] < 30), 1.0, 
                                     np.where((data['short_rsi'] > 70) & (data['medium_rsi'] > 70) & (data['long_rsi'] > 70), -1.0, 0.0))
        
        # Manteniamo la posizione fino al segnale opposto
        signals['positions'] = signals['signal']
        return signals
    

# Mean Reversion ATR Strategy
class MeanReversionATRStrategy(Strategy):
    def __init__(self, ma_period=20, atr_period=14, threshold=2.0):
        self.ma_period = ma_period
        self.atr_period = atr_period
        self.threshold = threshold

    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        data['MA'] = TechnicalIndicators.calculate_indicator('SMA', data['Close'], timeperiod=self.ma_period)
        data['ATR'] = TechnicalIndicators.calculate_indicator('ATR', data['High'], data['Low'], data['Close'], timeperiod=self.atr_period)

        signals['signal'] = np.where(data['Close'] < (data['MA'] - self.threshold * data['ATR']), 1.0, 
                                     np.where(data['Close'] > (data['MA'] + self.threshold * data['ATR']), -1.0, 0.0))
        
        # Manteniamo la posizione fino al ritorno verso la media
        signals['positions'] = signals['signal']
        return signals
    

# LSTM RSI MACD Strategy
class LSTMRsiMacdStrategy(Strategy):
    def __init__(self, lookback=60, rsi_period=14, macd_fast=12, macd_slow=26, macd_signal=9):
        self.lookback = lookback
        self.rsi_period = rsi_period
        self.macd_fast = macd_fast
        self.macd_slow = macd_slow
        self.macd_signal = macd_signal
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=True, input_shape=(self.lookback, 1)))
        model.add(LSTM(units=50))
        model.add(Dense(units=1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy')
        return model

    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(data[['Close']])
        x_train, y_train = [], []

        for i in range(self.lookback, len(scaled_data)):
            x_train.append(scaled_data[i-self.lookback:i, 0])
            y_train.append(1 if scaled_data[i, 0] > scaled_data[i-1, 0] else 0)
        
        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
        self.model.fit(x_train, y_train, epochs=5, batch_size=1, verbose=0)

        data['RSI'] = TechnicalIndicators.calculate_indicator('RSI', data['Close'], timeperiod=self.rsi_period)
        macd, macd_signal, _ = TechnicalIndicators.calculate_indicator('MACD', data['Close'], fastperiod=self.macd_fast, slowperiod=self.macd_slow, signalperiod=self.macd_signal)

        for i in range(self.lookback, len(scaled_data)):
            input_data = scaled_data[i-self.lookback:i, 0].reshape(1, -1, 1)
            prediction = self.model.predict(input_data, verbose=0)
            if prediction > 0.5 and data['RSI'].iloc[i] > 50 and macd[i] > macd_signal[i]:
                signals['signal'].iloc[i] = 1.0
            elif prediction <= 0.5 and data['RSI'].iloc[i] < 50 and macd[i] < macd_signal[i]:
                signals['signal'].iloc[i] = -1.0

        # La posizione segue il segnale predittivo
        signals['positions'] = signals['signal'].diff().fillna(0.0)
        return signals


# CNN and Candle Pattern Strategy
class CNNandCandlePatternStrategy(Strategy):
    def __init__(self, lookback=60):
        self.lookback = lookback
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(self.lookback, 1)))
        model.add(Flatten())
        model.add(Dense(units=50, activation='relu'))
        model.add(Dense(units=1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy')
        return model

    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(data[['Close']])
        x_train, y_train = [], []

        for i in range(self.lookback, len(scaled_data)):
            x_train.append(scaled_data[i-self.lookback:i, 0])
            y_train.append(1 if scaled_data[i, 0] > scaled_data[i-1, 0] else 0)

        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
        self.model.fit(x_train, y_train, epochs=5, batch_size=1, verbose=0)

        hammer_pattern = TechnicalIndicators.calculate_indicator('CDLHAMMER', data['Open'], data['High'], data['Low'], data['Close'])
        
        for i in range(self.lookback, len(scaled_data)):
            input_data = scaled_data[i-self.lookback:i, 0].reshape(1, -1, 1)
            prediction = self.model.predict(input_data, verbose=0)
            if prediction > 0.5 and hammer_pattern[i] != 0:
                signals['signal'].iloc[i] = 1.0
            elif prediction <= 0.5 and hammer_pattern[i] != 0:
                signals['signal'].iloc[i] = -1.0

        # La posizione segue il segnale predittivo
        signals['positions'] = signals['signal'].diff().fillna(0.0)
        return signals
    

# LSTM Bollinger Breakout Strategy
class LSTMBollingerBreakoutStrategy(Strategy):
    def __init__(self, lookback=60, bb_period=20, bb_dev=2):
        self.lookback = lookback
        self.bb_period = bb_period
        self.bb_dev = bb_dev
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=True, input_shape=(self.lookback, 1)))
        model.add(LSTM(units=50))
        model.add(Dense(units=1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy')
        return model

    def generate_signals(self, data):
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(data[['Close']])
        x_train, y_train = [], []

        for i in range(self.lookback, len(scaled_data)):
            x_train.append(scaled_data[i-self.lookback:i, 0])
            y_train.append(1 if scaled_data[i, 0] > scaled_data[i-1, 0] else 0)

        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
        self.model.fit(x_train, y_train, epochs=5, batch_size=1, verbose=0)

        upper_band, middle_band, lower_band = TechnicalIndicators.calculate_indicator(
            'BBANDS', data['Close'], timeperiod=self.bb_period, nbdevup=self.bb_dev, nbdevdn=self.bb_dev)

        for i in range(self.lookback, len(scaled_data)):
            input_data = scaled_data[i-self.lookback:i, 0].reshape(1, -1, 1)
            prediction = self.model.predict(input_data, verbose=0)
            if prediction > 0.5 and data['Close'].iloc[i] > upper_band[i]:
                signals['signal'].iloc[i] = 1.0  # Segnale di breakout
            elif prediction <= 0.5 and data['Close'].iloc[i] < lower_band[i]:
                signals['signal'].iloc[i] = -1.0  # Segnale di breakdown

        # La posizione segue il segnale predittivo
        signals['positions'] = signals['signal'].diff().fillna(0.0)
        return signals  


# IBSMeanReversionStrategy Strategy
class IBSMeanReversionStrategy:
    def __init__(self, rolling_mean_period=25, high_rolling_period=10, atr_multiplier=2.5, ibs_threshold=0.3, sma_period=200):
        self.rolling_mean_period = rolling_mean_period
        self.high_rolling_period = high_rolling_period
        self.atr_multiplier = atr_multiplier
        self.ibs_threshold = ibs_threshold
        self.sma_period = sma_period

    def generate_signals(self, data):
        """
        Generate trading signals based on the strategy rules.

        Parameters:
        - data: pd.DataFrame
            Historical price data with columns 'High', 'Low', 'Close'.

        Returns:
        - signals: pd.DataFrame
            DataFrame containing the 'signal' and 'positions' columns.
        """
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        # Calculate the rolling mean of High minus Low over the last 25 days
        data['HL_Rolling_Mean'] = (data['High'] - data['Low']).rolling(window=self.rolling_mean_period).mean()

        # Calculate the IBS indicator
        data['IBS'] = (data['Close'] - data['Low']) / (data['High'] - data['Low'])

        # Calculate the rolling High over the last 10 days
        data['High_Rolling'] = data['High'].rolling(window=self.high_rolling_period).max()

        # Calculate the lower band
        data['Lower_Band'] = data['High_Rolling'] - self.atr_multiplier * data['HL_Rolling_Mean']

        # Calculate the 200-day Simple Moving Average (SMA)
        data['200_SMA'] = data['Close'].rolling(window=self.sma_period).mean()

        # Determine market condition: Bull or Bear
        data['Market_Condition'] = np.where(data['Close'] > data['200_SMA'], 'Bull', 'Bear')

        # Generate signals
        for i in range(1, len(data)):
            if data['Market_Condition'].iloc[i] == 'Bull':
                if (data['Close'].iloc[i] < data['Lower_Band'].iloc[i]) and (data['IBS'].iloc[i] < self.ibs_threshold):
                    signals['signal'].iloc[i] = 1.0  # Buy signal
                elif data['Close'].iloc[i] > data['High'].iloc[i - 1]:
                    signals['signal'].iloc[i] = 0.0  # Sell signal
            else:
                signals['signal'].iloc[i] = 0.0  # No position in Bear market

        # Forward-fill the signal to maintain positions
        signals['positions'] = signals['signal']

        return signals   

strategy_list = [
    MovingAverageCrossStrategy(),
    RSIOverboughtOversoldStrategy(),
    BollingerBandsStrategy(),
    LSTMNeuralNetworkStrategy(),
    WeeklyRSIStrategy(),
    TurnOfTheMonthStrategy(),
    VolatilityBasedStrategy(),
    RussellRebalancingStrategy(),
    RubberBandStrategy(),
    MFIIndicatorStrategy(),
    RSIandBollingerBandsStrategy(),
    MultiTimeframeRSIStrategy(),
    MeanReversionATRStrategy(),
    LSTMRsiMacdStrategy(),
    CNNandCandlePatternStrategy(),
    LSTMBollingerBreakoutStrategy(),
    IBSMeanReversionStrategy(),
    BuyAndHold()
    ]

## **Backtesting: Definizione e Aspetti Tecnici**

### **Cos’è il Backtesting?**
Il backtesting è il processo di valutazione di una strategia di trading utilizzando dati storici per simulare la sua performance. L'obiettivo principale è verificare come la strategia si sarebbe comportata in passato, fornendo indicazioni sul suo potenziale futuro. 

Un backtest ben progettato può aiutare a:
- Valutare l'efficacia della strategia.
- Identificare punti deboli o aree di miglioramento.
- Confrontare diverse strategie prima di implementarle in tempo reale.

---

### **Componenti Chiave del Backtesting**

#### 1. **Dati Storici**
I dati storici sono il punto di partenza per qualsiasi backtest. Devono essere accurati e rappresentativi del mercato. Tipi di dati richiesti:
- **Prezzi:** Apertura (Open), Massimo (High), Minimo (Low), Chiusura (Close).
- **Volume:** Quantità di asset scambiati.
- **Dati Aggiuntivi:** Spread bid/ask, commissioni, dati macroeconomici, ecc.

#### 2. **Regole della Strategia**
Le regole della strategia specificano quando entrare, uscire o rimanere fuori dal mercato. Ad esempio:
- **Segnali di ingresso:** Criteri per acquistare o vendere.
- **Segnali di uscita:** Criteri per chiudere una posizione.
- **Gestione del rischio:** Livelli di stop loss, take profit, gestione del capitale.

#### 3. **Metodologia di Backtesting**
Ci sono due approcci principali:
- **Backtesting discreto:** Viene eseguito manualmente, analizzando i dati storici e applicando la strategia passo passo.
- **Backtesting algoritmico:** Utilizza codici per eseguire il backtest in modo automatico e sistematico.

#### 4. **Metriche di Performance**
Durante il backtest, è essenziale monitorare e calcolare metriche chiave, come:
- **Profitto Netto:** Guadagni totali meno perdite totali.
- **Drawdown Massimo:** La perdita percentuale massima dal picco al minimo.
- **Rapporto Rendimento/Rischio:** Valuta il rendimento medio per unità di rischio.
- **Sharpe Ratio:** Misura il rendimento corretto per la volatilità.
- **Win Rate:** Percentuale di trade vincenti rispetto al totale.

---

### **Passaggi del Processo di Backtesting**

1. **Preparazione dei Dati**
   - Importare dati storici accurati.
   - Pulire e formattare i dati per l'elaborazione (es. rimuovere dati mancanti o anomali).

2. **Implementazione della Strategia**
   - Scrivere le regole di ingresso, uscita e gestione del rischio in un codice o framework.

3. **Simulazione**
   - Applicare la strategia ai dati storici e calcolare i risultati per ogni operazione.

4. **Analisi dei Risultati**
   - Valutare le metriche di performance.
   - Identificare i periodi di forza e debolezza della strategia.

5. **Iterazione**
   - Modificare la strategia per migliorare i risultati.
   - Eseguire nuovamente il backtest per valutare i cambiamenti.

---

### **Sfide e Limitazioni**

#### 1. **Overfitting**
- Adattare troppo la strategia ai dati storici può portare a risultati eccellenti nel passato ma scarsi nel futuro.

#### 2. **Bias nei Dati**
- Dati inaccurati o non rappresentativi possono distorcere i risultati del backtest.

#### 3. **Costi di Trading**
- Commissioni, slippage e spread bid/ask devono essere inclusi per simulare realisticamente i profitti.

#### 4. **Condizioni di Mercato**
- Il mercato è dinamico, e condizioni passate potrebbero non ripetersi in futuro.

---

### **Best Practice nel Backtesting**
1. **Usare un Dataset Ampio**
   - Includere periodi di alta e bassa volatilità, bull e bear market.

2. **Includere Costi di Trading**
   - Considerare commissioni, spread e slippage per simulare condizioni realistiche.

3. **Dividere i Dati**
   - Separare i dati in set di training e test (es. dati in-sample e out-of-sample).

4. **Evita Overfitting**
   - Non ottimizzare eccessivamente la strategia sui dati storici.

5. **Stress Testing**
   - Testare la strategia in condizioni estreme o scenari improbabili.

---

### **Esempio Pratico**
Supponiamo di voler testare una strategia di crossover delle medie mobili.

### Parametri:
- **Media Mobile Breve:** 20 periodi.
- **Media Mobile Lunga:** 50 periodi.
- **Segnale di Acquisto:** Quando la media breve supera quella lunga.
- **Segnale di Vendita:** Quando la media breve scende sotto quella lunga.

#### Passaggi:
1. Importare i dati storici.
2. Calcolare le medie mobili.
3. Generare segnali basati sul crossover.
4. Simulare le operazioni.
5. Analizzare i risultati:
   - Calcolare profitto netto, drawdown massimo e Sharpe Ratio.

---

### **Conclusione**
Il backtesting è uno strumento essenziale per sviluppare e valutare strategie di trading. Tuttavia, richiede un approccio metodico per evitare errori e garantire risultati affidabili. Utilizzando dati accurati, metriche robuste e tecniche di simulazione realistiche, è possibile creare strategie più solide e adatte alle condizioni di mercato reali.

In [ ]:
from trading_system_py.analysis.backtest.backtest import Backtest
from trading_system_py.analysis.backtest.backtest_multiple_combination_portfolio import BacktestMultipleCombinationPortfolio

In [ ]:
strategy_combination = {
    'AAPL': MovingAverageCrossStrategy(), 
    'MSFT': MovingAverageCrossStrategy(), 
    'AMZN': RSIOverboughtOversoldStrategy(), 
    'GOOGL': BollingerBandsStrategy(), 
    'TSLA': BuyAndHold(), 
    'JNJ': RSIOverboughtOversoldStrategy(), 
    'PG': BuyAndHold(), 
    'KO': BuyAndHold(), 
    'XOM': RSIOverboughtOversoldStrategy(), 
    'JPM': BuyAndHold(), 
    'WMT': BollingerBandsStrategy(), 
    'V': LSTMNeuralNetworkStrategy(), 
    'MCD': BuyAndHold(), 
    'INTC': BuyAndHold(), 
    'LQD': MovingAverageCrossStrategy(), 
    'TLT': MovingAverageCrossStrategy(), 
    'BND': BuyAndHold(), 
    'EMB': LSTMNeuralNetworkStrategy(), 
    'HYG': BuyAndHold()
}

In [ ]:
# Inizializza il backtest con la combinazione di strategie
backtest = Backtest(portfolio=portfolio, strategy_combination=strategy_combination, verbose=True)

# Esegui il backtest
backtest.run()

# Visualizza la performance del portafoglio e il drawdown
backtest.plot_performance(add_drawdown=True, normalize=False)

In [ ]:
backtest.plot_performance(add_drawdown=True, normalize=True)

In [ ]:
backtest.plot_signals(isin='AAPL')

In [ ]:
pd.DataFrame(backtest.portfolio.transaction_history)

In [ ]:
pd.DataFrame(backtest.portfolio.portfolio_history)

In [ ]:
# Crea il dizionario 'data' come specificato
isin_list = ['isin_1', 'isin_2', 'isin_3']
data_dict = {k: {'History': generate_dummy_data(start_date, end_date, k)} for k in isin_list}

strategy_combination = {
    'isin_1': MovingAverageCrossStrategy(), 
    'isin_2': MovingAverageCrossStrategy(), 
    'isin_3': RSIOverboughtOversoldStrategy()
}

# Inizializza il backtest con la combinazione di strategie
portfolio = Portfolio(data=data_dict, init_cash=10000, verbose=False)
backtest_sim = Backtest(portfolio=portfolio, strategy_combination=strategy_combination, verbose=False)

# Esegui il backtest
backtest_sim.run(make_simulation=True, num_simulation=100)

In [ ]:
backtest_sim.simulation_results

In [ ]:
backtest_sim.plot_simulated_isin_trajectory(isin='isin_1')

In [ ]:
backtest_sim.plot_simulated_portfolio_results()

In [ ]:
strategy_multiple_combination = {
    'isin_1': [MovingAverageCrossStrategy(), RSIOverboughtOversoldStrategy()],
    'isin_2': [MovingAverageCrossStrategy(), BuyAndHold()], 
    'isin_3': [RSIOverboughtOversoldStrategy()], 
}

In [ ]:
# Crea il dizionario 'data' come specificato
isin_list = ['isin_1', 'isin_2', 'isin_3']
data_dict = {k: {'History': generate_dummy_data(start_date, end_date, k)} for k in isin_list}

# Inizializza il backtest con la combinazione di strategie
portfolio_fake = Portfolio(data=data_dict, init_cash=10000, verbose=False)
backtest_multi = BacktestMultipleCombinationPortfolio(portfolio=portfolio_fake, strategy_dict=strategy_multiple_combination, verbose=False)

# Esegui il backtest
backtest_multi.run()

In [ ]:
backtest_multi.plot_performance()

In [ ]:
class RiskManagement:
    def __init__(self, portfolio, risk_free_rate=0.01, confidence_level=0.95, window_size=252):
        """
        Inizializza la classe RiskManagement per valutare il rischio di un portafoglio.

        Parametri:
        - portfolio: Portfolio
            Oggetto Portfolio contenente storico delle transazioni e del valore del portafoglio.
        - risk_free_rate: float
            Tasso di rendimento privo di rischio per il calcolo dello Sharpe e Sortino ratio.
        - confidence_level: float
            Livello di confidenza per calcolare VaR e CVaR (default 95%).
        - window_size: int
            Dimensione della finestra per il calcolo rolling delle metriche.
        """
        self.portfolio = portfolio
        self.portfolio_returns = self._calculate_returns_from_history()  # Rendimenti calcolati dal valore del portafoglio
        self.risk_free_rate = risk_free_rate
        self.confidence_level = confidence_level
        self.window_size = window_size

    def _calculate_returns_from_history(self):
        """
        Calcola i rendimenti giornalieri dal valore del portafoglio.
        """
        portfolio_values = pd.Series([record['total_value'] for record in self.portfolio.portfolio_history],
                                     index=[record['date'] for record in self.portfolio.portfolio_history])
        returns = portfolio_values.pct_change().dropna()
        return returns

    def calculate_trade_results(self):
        """
        Calcola il numero di trade positivi e negativi a partire dal transaction_history del portafoglio.

        Ritorna:
        - positive_trades: int
            Numero di trade positivi.
        - negative_trades: int
            Numero di trade negativi.
        """
        positive_trades = 0
        negative_trades = 0

        for transaction in self.portfolio.transaction_history:
            if transaction['type'] == 'SELL' or transaction['type'] == 'COVER_SHORT':
                profit = transaction['profit']  # Viene già calcolato al momento della vendita/copertura
                if profit > 0:
                    positive_trades += 1
                else:
                    negative_trades += 1

        return positive_trades, negative_trades

    def calculate_var(self):
        var = np.percentile(self.portfolio_returns, 100 * (1 - self.confidence_level))
        return var

    def calculate_cvar(self):
        var = self.calculate_var()
        cvar = self.portfolio_returns[self.portfolio_returns <= var].mean()
        return cvar

    def calculate_sharpe_ratio(self):
        excess_returns = self.portfolio_returns - self.risk_free_rate / self.window_size
        sharpe_ratio = excess_returns.mean() / excess_returns.std()
        return sharpe_ratio

    def calculate_sortino_ratio(self):
        excess_returns = self.portfolio_returns - self.risk_free_rate / self.window_size
        downside_risk = np.sqrt((excess_returns[excess_returns < 0] ** 2).mean())
        sortino_ratio = excess_returns.mean() / downside_risk
        return sortino_ratio

    def calculate_max_drawdown(self):
        cumulative_returns = (1 + self.portfolio_returns).cumprod()
        max_drawdown = (cumulative_returns - cumulative_returns.cummax()).min()
        return max_drawdown

    def plot_trade_results(self):
        """
        Crea un grafico per visualizzare il numero di trade positivi e negativi.
        """
        positive_trades, negative_trades = self.calculate_trade_results()

        # Crea un grafico a barre per i risultati dei trade
        fig = go.Figure()

        fig.add_trace(go.Bar(x=['Positive Trades'], y=[positive_trades], name='Positive Trades', marker_color='green'))
        fig.add_trace(go.Bar(x=['Negative Trades'], y=[negative_trades], name='Negative Trades', marker_color='red'))

        fig.update_layout(
            title='Numero di Trade Positivi e Negativi',
            xaxis_title='Tipo di Trade',
            yaxis_title='Numero di Trade',
            hovermode='x unified'
        )

        fig.show()

    def plot_risk_metrics(self):
        var = self.calculate_var()
        cvar = self.calculate_cvar()

        fig = go.Figure()

        fig.add_trace(go.Scatter(x=self.portfolio_returns.index, 
                                 y=self.portfolio_returns.values, 
                                 mode='lines', 
                                 name='Portfolio Returns'))

        fig.add_trace(go.Scatter(x=self.portfolio_returns.index, 
                                 y=[var] * len(self.portfolio_returns), 
                                 mode='lines', 
                                 line=dict(color='red', dash='dash'),
                                 name=f'VaR ({self.confidence_level*100}%)'))

        fig.add_trace(go.Scatter(x=self.portfolio_returns.index, 
                                 y=[cvar] * len(self.portfolio_returns), 
                                 mode='lines', 
                                 line=dict(color='orange', dash='dash'),
                                 name=f'CVaR ({self.confidence_level*100}%)'))

        fig.update_layout(
            title='Risk Metrics: VaR and CVaR',
            xaxis_title='Date',
            yaxis_title='Returns',
            hovermode='x unified'
        )

        fig.show()

    def plot_sharpe_ratio(self):
        sharpe_ratio = self.calculate_sharpe_ratio()

        fig = go.Figure()

        fig.add_trace(go.Scatter(x=self.portfolio_returns.index, 
                                 y=[sharpe_ratio] * len(self.portfolio_returns), 
                                 mode='lines', 
                                 line=dict(color='blue', dash='dash'),
                                 name=f'Sharpe Ratio: {sharpe_ratio:.2f}'))

        fig.update_layout(
            title='Sharpe Ratio del Portafoglio',
            xaxis_title='Date',
            yaxis_title='Sharpe Ratio',
            hovermode='x unified'
        )

        fig.show()

    def plot_max_drawdown(self):
        cumulative_returns = (1 + self.portfolio_returns).cumprod()
        drawdown = cumulative_returns - cumulative_returns.cummax()
        max_drawdown = self.calculate_max_drawdown()

        fig = go.Figure()

        fig.add_trace(go.Scatter(x=cumulative_returns.index, 
                                 y=drawdown.values, 
                                 mode='lines', 
                                 line=dict(color='red'),
                                 name='Drawdown'))

        fig.add_trace(go.Scatter(x=cumulative_returns.index, 
                                 y=[max_drawdown] * len(cumulative_returns), 
                                 mode='lines', 
                                 line=dict(color='red', dash='dash'),
                                 name=f'Max Drawdown: {max_drawdown:.2%}'))

        fig.update_layout(
            title='Maximum Drawdown',
            xaxis_title='Date',
            yaxis_title='Drawdown',
            hovermode='x unified'
        )

        fig.show()

In [ ]:
risk_mgmt = RiskManagement(portfolio=portfolio)

In [ ]:
risk_mgmt.plot_risk_metrics()

In [ ]:
risk_mgmt.plot_trade_results()

In [ ]:
import optuna

class HyperparameterOptimization:
    def __init__(self, strategy_class, portfolio, hyperparameter_ranges):
        """
        Inizializza la classe per l'ottimizzazione degli iperparametri.

        Parametri:
        - strategy_class: tipo
            Classe della strategia da ottimizzare.
        - portfolio: Portfolio
            Oggetto Portfolio contenente dati e configurazioni.
        - hyperparameter_ranges: dict
            Dizionario con chiavi come nomi degli iperparametri e valori come tuple che definiscono il tipo e l'intervallo (min, max).
        """
        self.strategy_class = strategy_class
        self.portfolio = portfolio
        self.hyperparameter_ranges = hyperparameter_ranges

    def objective(self, trial):
        """
        Funzione obiettivo per Optuna che massimizza il rendimento del portafoglio.

        Parametri:
        - trial: optuna.Trial
            Oggetto Trial di Optuna che genera parametri suggeriti.

        Ritorna:
        - rendimento finale del portafoglio.
        """
        # Dizionario per memorizzare gli iperparametri suggeriti
        suggested_params = {}

        # Itera sugli iperparametri e i loro intervalli
        for param_name, (param_type, param_range) in self.hyperparameter_ranges.items():
            if param_type == 'int':
                suggested_params[param_name] = trial.suggest_int(param_name, param_range[0], param_range[1])
            elif param_type == 'float':
                suggested_params[param_name] = trial.suggest_float(param_name, param_range[0], param_range[1])
            elif param_type == 'categorical':
                suggested_params[param_name] = trial.suggest_categorical(param_name, param_range)
            else:
                raise ValueError(f"Tipo di parametro non supportato: {param_type}")

        # Istanzia la strategia con gli iperparametri suggeriti
        strategy = self.strategy_class(**suggested_params)
        
        # Esegui il backtest con il portfolio corrente
        backtest = Backtest(portfolio=self.portfolio, strategy_combination={'isin_1': strategy})
        backtest.run(make_simulation=False)
        
        # Calcola il rendimento finale del portafoglio
        portfolio_values = [record['total_value'] for record in backtest.portfolio.portfolio_history]
        initial_cash = self.portfolio.init_cash
        total_return = (portfolio_values[-1] - initial_cash) / initial_cash
        
        return total_return

    def optimize(self, n_trials=100):
        """
        Esegue l'ottimizzazione degli iperparametri.

        Parametri:
        - n_trials: int
            Numero di tentativi di ottimizzazione.

        Ritorna:
        - study.best_params: dict
            I migliori parametri trovati da Optuna.
        """
        study = optuna.create_study(direction='maximize')
        study.optimize(self.objective, n_trials=n_trials)
        
        print("Migliori iperparametri:", study.best_params)
        print("Miglior rendimento del portafoglio:", study.best_value)
        return study.best_params, study.best_value

In [ ]:
# Definizione degli intervalli di iperparametri
hyperparameter_ranges = {
    'short_window': ('int', (5, 50)),
    'long_window': ('int', (20, 200))
}

# Esegui l'ottimizzazione sugli iperparametri
optimizer = HyperparameterOptimization(MovingAverageCrossStrategy, portfolio, hyperparameter_ranges)
best_params, best_return = optimizer.optimize(n_trials=50)